In [1]:
########Settings to shutup tensorflow###########################################################################
# WARNING:tensorflow:Entity <..>> could not be transformed and will be executed as-is
# to stop this warning, downgrade gast
# pip3 install gast==0.2.2

import os, warnings
warnings.filterwarnings('ignore') # stop future warnings

import tensorflow as tf
# even after all this, deprication warning will appear
# no way to stop this thn to prevent all messages, except errors
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# this is dangerous, make sure that only deprication warnings are being issued before executing above line

os.environ['CUDA_VISIBLE_DEVICES'] = '' # required for tf to use gpu
os.environ['TF_XLA_FLAGS']='--tf_xla_auto_jit=0 /home/raghavan/pinn_test' # accelerate tf models, stops warning
###############################################################################################################

In [2]:
def log(s, file):
    if not file:
        print(s)
    else:
        file.write(str(s)+'\n')
def getxyz(atoms, step, f=None):
    pos = atoms.get_positions()
    symb = atoms.get_chemical_symbols()
    d = [str(s)+" "+str(i[0])+" "+str(i[1])+" "+str(i[2]) for s, i in zip(symb,pos)]
    
    log(len(d), f)
    log("Step: "+str(step), f)
    for r in d:
        log(r, f)

In [3]:
import MDAnalysis as mda
u = mda.Universe('data/mimic.trr')
u.atoms.positions

array([[12.7116  , 12.751599, 12.5616  ],
       [11.9716  , 11.981601, 11.501599],
       [13.211599, 11.961599, 13.7516  ],
       ...,
       [17.1416  ,  8.9916  , 24.4316  ],
       [17.521599,  8.5816  , 23.6016  ],
       [16.6416  ,  9.8216  , 24.191599]], dtype=float32)

In [4]:
import sys; sys.path.insert(0, '/home/raghavan/bin/mimicpy')
from mimicpy.parsers.mpt import MPT
elems = "".join([i for i in MPT.fromFile('data/mimic.mpt')['element']])
elems

{file} found!


'CCCOHHHHHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOHHOH

In [5]:
from ase import Atoms
atoms = Atoms(elems, positions=u.atoms.positions)

In [6]:
atoms.get_positions()

array([[12.7116003 , 12.75159931, 12.56159973],
       [11.97159958, 11.98160076, 11.50159931],
       [13.21159935, 11.96159935, 13.75160027],
       ...,
       [17.14159966,  8.99160004, 24.43160057],
       [17.52159882,  8.58160019, 23.60160065],
       [16.64159966,  9.82159996, 24.19159889]])

In [11]:
import sys; sys.path.insert(0, '/home/raghavan/bin/pinn')
from pinn.models import potential_model
from pinn.calculator import PiNN_calc
calc = PiNN_calc(potential_model('model'))
atoms.set_calculator(calc)

In [30]:
from ase.optimize import BFGS
from ase.io.trajectory import Trajectory
dyn = BFGS(atoms)
traj = Trajectory('pred/minim.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(fmax=0.05)

      Step     Time          Energy         fmax
BFGS:    0 10:01:06     -235.374451        3.2737
BFGS:    1 10:01:11     -255.027618        3.4132
BFGS:    2 10:01:17     -275.471497        3.5160
BFGS:    3 10:01:22     -296.994781        3.6154
BFGS:    4 10:01:27     -319.552856        3.6848
BFGS:    5 10:01:33     -343.553040        3.7034
BFGS:    6 10:01:38     -369.510986        3.6729
BFGS:    7 10:01:43     -396.453430        3.7285
BFGS:    8 10:01:48     -424.270325        3.8043
BFGS:    9 10:01:54     -453.227783        3.8472
BFGS:   10 10:01:59     -483.589752        3.8640
BFGS:   11 10:02:05     -515.610718        3.8618
BFGS:   12 10:02:10     -549.122131        3.8464
BFGS:   13 10:02:15     -582.056763        3.8231
BFGS:   14 10:02:21     -612.244568        4.0946
BFGS:   15 10:02:26     -639.792053        4.3946
BFGS:   16 10:02:32     -665.014709        4.6722
BFGS:   17 10:02:37     -688.426941        4.9102
BFGS:   18 10:02:43     -710.696045        5.0823
B

KeyboardInterrupt: 

In [37]:
traj = Trajectory('pred/minim.traj')
f = open('pred/minim.xyz', 'w')
for i, atoms in enumerate(traj):
    getxyz(atoms, i, f)

In [12]:
from ase.md.verlet import VelocityVerlet
from ase import units
from ase.md import MDLogger
from ase import units
from ase.io.trajectory import Trajectory

dyn = VelocityVerlet(atoms, dt=2*units.fs)
dyn.attach(MDLogger(dyn, atoms, '-', header=True, stress=False,
           peratom=True, mode="w"), interval=10)
traj = Trajectory('pred/md.traj', 'w', atoms)
dyn.attach(traj.write, interval=1)
dyn.run(100)
traj.close()

Time[ps]    Etot/N[eV]   Epot/N[eV]   Ekin/N[eV]    T[K]
0.0000          -1.9237      -2.4205       0.4967  3843.0
0.0200          -1.9189      -2.3864       0.4675  3616.7
0.0400          -1.9233      -2.4128       0.4894  3786.2
0.0600          -1.9186      -2.4336       0.5150  3984.3
0.0800          -1.9171      -2.4662       0.5491  4248.0
0.1000          -1.9173      -2.4536       0.5363  4149.0
0.1200          -1.9151      -2.4511       0.5360  4146.5
0.1400          -1.9142      -2.4642       0.5500  4255.3
0.1600          -1.9005      -2.4552       0.5547  4291.7
0.1800          -1.8948      -2.4583       0.5636  4360.2
0.2000          -1.8905      -2.4437       0.5533  4280.3


In [13]:
from ase.io.trajectory import Trajectory
traj = Trajectory('pred/md.traj')
xyz = open('pred/md.xyz', 'w')
for i, atoms in enumerate(traj):
    getxyz(atoms, i, xyz)

In [36]:
import ase
ase.io.proteindatabank.write_proteindatabank("md/md.pdb", traj[100])